In [46]:
import pandas as pd
import numpy as np
import wbgapi as wb
import yfinance as yf

from statsmodels.formula.api import ols
import statsmodels.api as sm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Overall Rules

- Refrain from saving datasets locally. You may experiment with your answers on a locally saved version of the datasets, but do not upload your local files with your homework as the datasets are very large. In your submitted answers datasets should be read from the original source URL.
- Document all of your steps by writing appropriate markdown cells in your notebook. Refrain from using code comments to explain what has been done.
- Avoid duplicating code. Do not copy and paste code from one cell to another. If copying and pasting is necessary, write a suitable function for the task at hand and call that function.
- Document your use of LLMs (ChatGPT, Claude, Code Pilot etc). Either take screenshots of your steps and include them with this notebook, or give me a full log (both questions and answers) in a markdown file named HW2-LLM-LOG.md.

Failure to adhere to these guidelines will result in a 25-point deduction for each infraction.

# HW2

## Q1

There are 22 countries surrounding the Mediterranean Sea: Spain, France, Monaco, Italy, Slovenia, Croatia, Bosnia and Herzegovina, Montenegro, Albania, Greece, Turkey, Syria, Lebanon, Israel, Palestine, Egypt, Libya, Tunisia, Algeria, and Morocco, with two island countries Malta and Cyprus.

1. Get the following data for every country in the list above from the World Bank Data server (using the `wbgapi` library)

- Adult female literacy (SE.ADT.LITR.FE.ZS)
- Adult female workforce participation rate (SL.TLF.ACTI.ZS)
- Child mortality rate (SP.DYN.IMRT.IN)
- Gini index (SI.POV.GINI)
- Life expectancy (SP.DYN.LE00.IN)
- GDP (NY.GDP.PCAP.CD)

2. Write a function that does linear regression for Log(mortality) against the other variables (except mortality).
3. Analyze the regression results for Spain, France, Turkey, Syria, and Israel.
4. Analyze the results for 2 other countries of your choice.

## Answers

Let us start by getting the data:

In [69]:
countries = ['ESP','FRA','MCO','ITA','HRV','BIH','MNE','ALB','GRC','TUR','SYR','LBN',
             'ISR','PSE','EGY','LBY','TUN','DZA','MAR','MLT','CYP']

series = {'Literacy': 'SE.ADT.LITR.FE.ZS',
          'Participation': 'SL.TLF.ACTI.ZS',
          'Mortality': 'SP.DYN.IMRT.IN',
          'Expectancy': 'SP.DYN.LE00.IN',
          'GINI': 'SI.POV.GINI',
          'GDP': 'NY.GDP.PCAP.CD'}


data = wb.data.DataFrame(list(series.values()), economy=countries).T
data

economy            ALB                                               \
series  NY.GDP.PCAP.CD SE.ADT.LITR.FE.ZS SI.POV.GINI SL.TLF.ACTI.ZS   
YR1960             NaN               NaN         NaN            NaN   
YR1961             NaN               NaN         NaN            NaN   
YR1962             NaN               NaN         NaN            NaN   
YR1963             NaN               NaN         NaN            NaN   
YR1964             NaN               NaN         NaN            NaN   
...                ...               ...         ...            ...   
YR2019     5396.214243               NaN        30.1         70.373   
YR2020     5343.037704               NaN        29.4         67.363   
YR2021     6377.203096               NaN         NaN         68.684   
YR2022     6810.114041         98.300003         NaN         71.367   
YR2023             NaN               NaN         NaN            NaN   

economy                                          BIH                    \
series  SP.DYN.IMRT.IN SP.DYN.LE00.IN NY.GDP.PCAP.CD SE.ADT.LITR.FE.ZS   
YR1960             NaN         54.439            NaN               NaN   
YR1961             NaN         55.634            NaN               NaN   
YR1962             NaN         56.671            NaN               NaN   
YR1963             NaN         57.844            NaN               NaN   
YR1964             NaN         58.983            NaN               NaN   
...                ...            ...            ...               ...   
YR2019             8.4         79.282    6094.724823               NaN   
YR2020             8.4         76.989    6095.104237               NaN   
YR2021             8.4         76.463    7230.198838               NaN   
YR2022             NaN            NaN    7568.798480         97.099998   
YR2023             NaN            NaN            NaN               NaN   

economy                             ...         TUN                 \
series  SI.POV.GINI SL.TLF.ACTI.ZS  ... SI.POV.GINI SL.TLF.ACTI.ZS   
YR1960          NaN            NaN  ...         NaN            NaN   
YR1961          NaN            NaN  ...         NaN            NaN   
YR1962          NaN            NaN  ...         NaN            NaN   
YR1963          NaN            NaN  ...         NaN            NaN   
YR1964          NaN            NaN  ...         NaN            NaN   
...             ...            ...  ...         ...            ...   
YR2019          NaN         57.265  ...         NaN         51.891   
YR2020          NaN         57.784  ...         NaN         49.715   
YR2021          NaN         60.999  ...        33.7         50.037   
YR2022          NaN         61.552  ...         NaN         52.348   
YR2023          NaN            NaN  ...         NaN            NaN   

economy                                          TUR                    \
series  SP.DYN.IMRT.IN SP.DYN.LE00.IN NY.GDP.PCAP.CD SE.ADT.LITR.FE.ZS   
YR1960             NaN         43.940     275.041699               NaN   
YR1961             NaN         44.146     282.742464               NaN   
YR1962           181.7         45.513     307.306286               NaN   
YR1963           173.2         46.500     347.177091               NaN   
YR1964           165.0         47.365     365.133869               NaN   
...                ...            ...            ...               ...   
YR2019            14.5         75.993    9215.440875         94.424042   
YR2020            14.3         75.292    8638.739133               NaN   
YR2021            14.0         73.772    9743.213131               NaN   
YR2022             NaN            NaN   10674.504173               NaN   
YR2023             NaN            NaN            NaN               NaN   

economy                                                           
series  SI.POV.GINI SL.TLF.ACTI.ZS SP.DYN.IMRT.IN SP.DYN.LE00.IN  
YR1960          NaN            NaN          171.5         50.740  
YR1961          NaN            NaN    

For the models, I am going to rename the columns, interpolate the missing values and then backfill the rest.

In [70]:
data.rename(columns={v: k for (k,v) in series.items()},inplace=True)
data.interpolate(inplace=True)
data.bfill(inplace=True)
data

economy          ALB                                                      \
series           GDP   Literacy  GINI Participation Mortality Expectancy   
YR1960    639.484730  98.252274  27.0        69.274      76.7     54.439   
YR1961    639.484730  98.252274  27.0        69.274      76.7     55.634   
YR1962    639.484730  98.252274  27.0        69.274      76.7     56.671   
YR1963    639.484730  98.252274  27.0        69.274      76.7     57.844   
YR1964    639.484730  98.252274  27.0        69.274      76.7     58.983   
...              ...        ...   ...           ...       ...        ...   
YR2019   5396.214243  97.650823  30.1        70.373       8.4     79.282   
YR2020   5343.037704  97.867216  29.4        67.363       8.4     76.989   
YR2021   6377.203096  98.083610  29.4        68.684       8.4     76.463   
YR2022   6810.114041  98.300003  29.4        71.367       8.4     76.463   
YR2023   6810.114041  98.300003  29.4        71.367       8.4     76.463   

economy          BIH                                 ...    TUN                \
series           GDP   Literacy  GINI Participation  ...   GINI Participation   
YR1960    333.783179  81.959328  30.0        49.564  ...  43.40        50.628   
YR1961    333.783179  81.959328  30.0        49.564  ...  43.40        50.628   
YR1962    333.783179  81.959328  30.0        49.564  ...  43.40        50.628   
YR1963    333.783179  81.959328  30.0        49.564  ...  43.40        50.628   
YR1964    333.783179  81.959328  30.0        49.564  ...  43.40        50.628   
...              ...        ...   ...           ...  ...    ...           ...   
YR2019   6094.724823  96.370163  33.0        57.265  ...  33.40        51.891   
YR2020   6095.104237  96.613441  33.0        57.784  ...  33.55        49.715   
YR2021   7230.198838  96.856720  33.0        60.999  ...  33.70        50.037   
YR2022   7568.798480  97.099998  33.0        61.552  ...  33.70        52.348   
YR2023   7568.798480  97.099998  33.0        61.552  ...  33.70        52.348   

economy                                TUR                                 \
series  Mortality Expectancy           GDP   Literacy  GINI Participation   
YR1960      181.7     43.940    275.041699  45.098919  43.5        60.068   
YR1961      181.7     44.146    282.742464  45.098919  43.5        60.068   
YR1962      181.7     45.513    307.306286  45.098919  43.5        60.068   
YR1963      173.2     46.500    347.177091  45.098919  43.5        60.068   
YR1964      165.0     47.365    365.133869  45.098919  43.5        60.068   
...           ...        ...           ...        ...   ...           ...   
YR2019       14.5     75.993   9215.440875  94.424042  43.8        57.710   
YR2020       14.3     75.292   8638.739133  94.424042  43.0        54.048   
YR2021       14.0     73.772   9743.213131  94.424042  44.4        56.321   
YR2022       14.0     73.772  10674.504173  94.424042  44.4        58.314   
YR2023       14.0     73.772  10674.504173  94.424042  44.4        58.314   

economy                       
series  Mortality Expectancy  
YR1960      171.5     50.740  
YR1961      166.2     51.550  
YR1962      160.8     52.382  
YR1963      155.5     53.173  
YR1964      150.4     53.714  
...           ...        ...  
YR2019        8.6     77.832  
YR2020        8.1     75.850  
YR2021        7.7     76.032  
YR2022        7.7     76.032  
YR2023        7.7     76.032  

[64 rows x 126 columns]

In [71]:
def experiment(data,country, formula):
    tmp = data[country]
    tmp['Mortality'] = np.log(tmp['Mortality'])
    res = ols(formula, data=tmp).fit()
    return res

### Turkey

Let us fit a model using the same order given in the original data frame.

In [72]:
res = experiment(data,'TUR','Mortality ~ GDP + Literacy + GINI + Participation + Expectancy')
res.summary()
sm.stats.anova_lm(res)

/tmp/ipykernel_274264/2751998543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Mortality'] = np.log(tmp['Mortality'])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Mortality   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     605.2
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           1.10e-48
Time:                        21:35:11   Log-Likelihood:                 37.721
No. Observations:                  64   AIC:                            -63.44
Df Residuals:                      58   BIC:                            -50.49
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        14.0203      3.204      4.376      0.000       7.607      20.433
GDP           -7.343e-05   1.08e-05     -6.805      0.000    -9.5e-05   -5.18e-05
Literacy         -0.0271      0.005     -5.690      0.000      -0.037      -0.018
GINI             -0.1296      0.073     -1.766      0.083      -0.277       0.017
Participation    -0.0106      0.007     -1.533      0.131      -0.024       0.003
Expectancy       -0.0276      0.009     -3.015      0.004      -0.046      -0.009
==============================================================================
Omnibus:                        9.959   Durbin-Watson:                   0.330
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               11.233
Skew:                          -0.683   Prob(JB):                      0.00364
Kurtosis:                       4.532   Cond. No.                     1.08e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.08e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

,df,sum_sq,mean_sq,F,PR(>F)
GDP,1.0,54.158242,54.158242,2724.721492,1.873110e-50
Literacy,1.0,5.732678,5.732678,288.413179,3.530806e-24
GINI,1.0,0.060456,0.060456,3.041582,8.645478e-02
Participation,1.0,0.013322,0.013322,0.670253,4.163145e-01
Expectancy,1.0,0.180722,0.180722,9.092188,3.804982e-03
Residual,58.0,1.152844,0.019877,NaN,NaN


When you change the order of the variables, their effect on the variation changes. Compare the previous model with this one.

In [73]:
res = experiment(data,'TUR','Mortality ~ Literacy + Participation + Expectancy + GDP + GINI')
res.summary()
sm.stats.anova_lm(res)

/tmp/ipykernel_274264/2751998543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Mortality'] = np.log(tmp['Mortality'])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Mortality   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     605.2
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           1.10e-48
Time:                        21:35:12   Log-Likelihood:                 37.721
No. Observations:                  64   AIC:                            -63.44
Df Residuals:                      58   BIC:                            -50.49
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        14.0203      3.204      4.376      0.000       7.607      20.433
Literacy         -0.0271      0.005     -5.690      0.000      -0.037      -0.018
Participation    -0.0106      0.007     -1.533      0.131      -0.024       0.003
Expectancy       -0.0276      0.009     -3.015      0.004      -0.046      -0.009
GDP           -7.343e-05   1.08e-05     -6.805      0.000    -9.5e-05   -5.18e-05
GINI             -0.1296      0.073     -1.766      0.083      -0.277       0.017
==============================================================================
Omnibus:                        9.959   Durbin-Watson:                   0.330
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               11.233
Skew:                          -0.683   Prob(JB):                      0.00364
Kurtosis:                       4.532   Cond. No.                     1.08e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.08e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

,df,sum_sq,mean_sq,F,PR(>F)
Literacy,1.0,59.067875,59.067875,2971.726998,1.588988e-51
Participation,1.0,0.000011,0.000011,0.000535,9.816314e-01
Expectancy,1.0,0.038496,0.038496,1.936754,1.693359e-01
GDP,1.0,0.977064,0.977064,49.156449,2.807273e-09
GINI,1.0,0.061974,0.061974,3.117960,8.269686e-02
Residual,58.0,1.152844,0.019877,NaN,NaN


After a few tries you'll find that for Turkey the following model has the best $R^2$ score. This indicates that the factor that effects the child mortality in Turkey is mothers' literacy and GDP. The literacy has statistically verifiable negative effect, so does the GDP as you can see below.

In [74]:
res = experiment(data,'TUR','Mortality ~ Literacy + GDP')
res.summary()
sm.stats.anova_lm(res)

/tmp/ipykernel_274264/2751998543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Mortality'] = np.log(tmp['Mortality'])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Mortality   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.976
Method:                 Least Squares   F-statistic:                     1298.
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           1.02e-50
Time:                        21:35:13   Log-Likelihood:                 31.338
No. Observations:                  64   AIC:                            -56.68
Df Residuals:                      61   BIC:                            -50.20
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.7094      0.127     52.677      0.000       6.455       6.964
Literacy      -0.0391      0.002    -15.763      0.000      -0.044      -0.034
GDP        -6.642e-05   1.11e-05     -5.973      0.000   -8.87e-05   -4.42e-05
==============================================================================
Omnibus:                        2.461   Durbin-Watson:                   0.136
Prob(Omnibus):                  0.292   Jarque-Bera (JB):                2.168
Skew:                          -0.343   Prob(JB):                        0.338
Kurtosis:                       2.415   Cond. No.                     3.99e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.99e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

,df,sum_sq,mean_sq,F,PR(>F)
Literacy,1.0,59.067875,59.067875,2560.240750,1.586878e-51
GDP,1.0,0.823044,0.823044,35.674075,1.298610e-07
Residual,61.0,1.407344,0.023071,NaN,NaN


### Spain

Let us check what happens with Spain:

In [75]:
res = experiment(data,'ESP','Mortality ~ GDP + Literacy + GINI + Participation + Expectancy')
res.summary()
sm.stats.anova_lm(res)

/tmp/ipykernel_274264/2751998543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Mortality'] = np.log(tmp['Mortality'])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Mortality   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     1597.
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           9.39e-61
Time:                        21:35:15   Log-Likelihood:                 71.335
No. Observations:                  64   AIC:                            -130.7
Df Residuals:                      58   BIC:                            -117.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        19.1006      1.168     16.356      0.000      16.763      21.438
GDP           -9.168e-06   4.74e-06     -1.933      0.058   -1.87e-05    3.24e-07
Literacy         -0.0421      0.012     -3.447      0.001      -0.067      -0.018
GINI              0.0175      0.012      1.512      0.136      -0.006       0.041
Participation     0.0398      0.005      7.716      0.000       0.029       0.050
Expectancy       -0.2084      0.008    -26.033      0.000      -0.224      -0.192
==============================================================================
Omnibus:                        1.603   Durbin-Watson:                   0.981
Prob(Omnibus):                  0.449   Jarque-Bera (JB):                1.584
Skew:                           0.357   Prob(JB):                        0.453
Kurtosis:                       2.711   Cond. No.                     2.05e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

,df,sum_sq,mean_sq,F,PR(>F)
GDP,1.0,47.465493,47.465493,6827.219391,7.243122e-62
Literacy,1.0,2.977515,2.977515,428.272069,1.840223e-28
GINI,1.0,0.002050,0.002050,0.294799,5.892424e-01
Participation,1.0,0.369093,0.369093,53.088614,9.684905e-10
Expectancy,1.0,4.711790,4.711790,677.722331,1.097174e-33
Residual,58.0,0.403239,0.006952,NaN,NaN


In [76]:
res = experiment(data,'TUR','Mortality ~ Literacy + GDP')
res.summary()
sm.stats.anova_lm(res)

/tmp/ipykernel_274264/2751998543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Mortality'] = np.log(tmp['Mortality'])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Mortality   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.976
Method:                 Least Squares   F-statistic:                     1298.
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           1.02e-50
Time:                        21:35:15   Log-Likelihood:                 31.338
No. Observations:                  64   AIC:                            -56.68
Df Residuals:                      61   BIC:                            -50.20
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.7094      0.127     52.677      0.000       6.455       6.964
Literacy      -0.0391      0.002    -15.763      0.000      -0.044      -0.034
GDP        -6.642e-05   1.11e-05     -5.973      0.000   -8.87e-05   -4.42e-05
==============================================================================
Omnibus:                        2.461   Durbin-Watson:                   0.136
Prob(Omnibus):                  0.292   Jarque-Bera (JB):                2.168
Skew:                          -0.343   Prob(JB):                        0.338
Kurtosis:                       2.415   Cond. No.                     3.99e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.99e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

,df,sum_sq,mean_sq,F,PR(>F)
Literacy,1.0,59.067875,59.067875,2560.240750,1.586878e-51
GDP,1.0,0.823044,0.823044,35.674075,1.298610e-07
Residual,61.0,1.407344,0.023071,NaN,NaN


The same model works pretty well for Spain too. Let us try the others:

### France

In [77]:
res = experiment(data,'FRA','Mortality ~ GDP + Literacy + GINI + Participation + Expectancy')
res.summary()
sm.stats.anova_lm(res)

/tmp/ipykernel_274264/2751998543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Mortality'] = np.log(tmp['Mortality'])


ValueError: zero-size array to reduction operation maximum which has no identity

We don't have enough data for France. 

### Israel

Let us move to Israel:


In [68]:
wb.data.DataFrame(list(series.values()), economy='ISR').T

series,NY.GDP.PCAP.CD,SE.ADT.LITR.FE.ZS,SI.POV.GINI,SL.TLF.ACTI.ZS,SP.DYN.IMRT.IN,SP.DYN.LE00.IN
YR1960,1229.174748,NaN,NaN,NaN,34.4,NaN
YR1961,1436.384439,NaN,NaN,NaN,32.7,72.006585
YR1962,1094.635848,NaN,NaN,NaN,31.0,72.112195
YR1963,1257.811405,NaN,NaN,NaN,29.5,NaN
YR1964,1375.892256,NaN,NaN,NaN,28.2,NaN
...,...,...,...,...,...,...
YR2019,44452.232562,NaN,38.3,73.200,2.8,82.804878
YR2020,44846.791595,NaN,37.8,71.513,2.8,82.648780
YR2021,52129.515961,NaN,37.9,71.649,2.7,82.500000
YR2022,54930.938808,NaN,NaN,73.337,NaN,NaN


In [78]:
res = experiment(data,'ISR','Mortality ~ GDP + Literacy + GINI + Participation + Expectancy')
res.summary()
sm.stats.anova_lm(res)

/tmp/ipykernel_274264/2751998543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Mortality'] = np.log(tmp['Mortality'])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Mortality   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                     825.8
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           4.80e-51
Time:                        21:35:23   Log-Likelihood:                 50.750
No. Observations:                  64   AIC:                            -91.50
Df Residuals:                      59   BIC:                            -80.71
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.0018      0.000     12.102      0.000       0.002       0.002
GDP           -1.015e-05   4.06e-06     -2.498      0.015   -1.83e-05   -2.02e-06
Literacy          0.1618      0.013     12.102      0.000       0.135       0.188
GINI             -0.0116      0.014     -0.838      0.406      -0.039       0.016
Participation     0.0403      0.016      2.463      0.017       0.008       0.073
Expectancy       -0.1862      0.015    -12.109      0.000      -0.217      -0.155
==============================================================================
Omnibus:                       12.901   Durbin-Watson:                   0.452
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               14.595
Skew:                           0.900   Prob(JB):                     0.000677
Kurtosis:                       4.494   Cond. No.                     2.75e+20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.7e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

,df,sum_sq,mean_sq,F,PR(>F)
GDP,1.0,38.640120,38.640120,2971.331109,3.589046e-52
Literacy,1.0,0.193660,0.193660,14.891992,2.846171e-04
GINI,1.0,2.344475,2.344475,180.284409,1.367763e-19
Participation,1.0,0.036959,0.036959,2.842042,9.710960e-02
Expectancy,1.0,1.870223,1.870223,143.815612,1.849653e-17
Residual,59.0,0.767254,0.013004,NaN,NaN


Israel is an anomaly: While for the other countries we looked (Turkey, Spain) female literacy was the most important factor reducing child mortality, for Israel, it was GDP and life expectancy.

In [93]:
res = experiment(data,'ISR','Mortality ~ GDP + Expectancy')
res.summary()
sm.stats.anova_lm(res)

/tmp/ipykernel_274264/2751998543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Mortality'] = np.log(tmp['Mortality'])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Mortality   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     1546.
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           5.56e-53
Time:                        21:40:12   Log-Likelihood:                 47.618
No. Observations:                  64   AIC:                            -89.24
Df Residuals:                      61   BIC:                            -82.76
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     16.4501      0.760     21.642      0.000      14.930      17.970
GDP        -4.943e-06   2.93e-06     -1.688      0.097   -1.08e-05    9.14e-07
Expectancy    -0.1845      0.011    -17.478      0.000      -0.206      -0.163
==============================================================================
Omnibus:                       17.574   Durbin-Watson:                   0.367
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.971
Skew:                           1.071   Prob(JB):                     6.23e-06
Kurtosis:                       5.097   Cond. No.                     1.22e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.22e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

,df,sum_sq,mean_sq,F,PR(>F)
GDP,1.0,38.640120,38.640120,2785.604871,1.281236e-52
Expectancy,1.0,4.237413,4.237413,305.479361,1.971118e-25
Residual,61.0,0.846153,0.013871,NaN,NaN


### Italy

Let us check Italy:

In [94]:
res = experiment(data,'ITA','Mortality ~ GDP + Literacy + GINI + Participation + Expectancy')
res.summary()
sm.stats.anova_lm(res)

/tmp/ipykernel_274264/2751998543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Mortality'] = np.log(tmp['Mortality'])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Mortality   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     1339.
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           1.51e-58
Time:                        21:44:04   Log-Likelihood:                 65.388
No. Observations:                  64   AIC:                            -118.8
Df Residuals:                      58   BIC:                            -105.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        23.6509      4.020      5.884      0.000      15.605      31.697
GDP            6.242e-06   3.99e-06      1.566      0.123   -1.74e-06    1.42e-05
Literacy         -0.0862      0.053     -1.619      0.111      -0.193       0.020
GINI              0.0350      0.012      2.837      0.006       0.010       0.060
Participation     0.0245      0.009      2.704      0.009       0.006       0.043
Expectancy       -0.2071      0.012    -16.954      0.000      -0.232      -0.183
==============================================================================
Omnibus:                        2.387   Durbin-Watson:                   0.878
Prob(Omnibus):                  0.303   Jarque-Bera (JB):                1.900
Skew:                          -0.421   Prob(JB):                        0.387
Kurtosis:                       3.072   Cond. No.                     7.90e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.9e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

,df,sum_sq,mean_sq,F,PR(>F)
GDP,1.0,51.800974,51.800974,6187.034067,1.228046e-60
Literacy,1.0,0.690940,0.690940,82.524843,9.628506e-13
GINI,1.0,0.564973,0.564973,67.479603,2.678090e-11
Participation,1.0,0.575313,0.575313,68.714513,2.008053e-11
Expectancy,1.0,2.406485,2.406485,287.427142,3.836137e-24
Residual,58.0,0.485605,0.008373,NaN,NaN


It seems Italy is a little different.

In [106]:
res = experiment(data,'ITA','Mortality ~ Literacy + Expectancy')
res.summary()
sm.stats.anova_lm(res)

/tmp/ipykernel_274264/2751998543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Mortality'] = np.log(tmp['Mortality'])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Mortality   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     2968.
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           1.69e-61
Time:                        22:30:27   Log-Likelihood:                 59.978
No. Observations:                  64   AIC:                            -114.0
Df Residuals:                      61   BIC:                            -107.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.8951      2.351      5.911      0.000       9.194      18.596
Literacy       0.0543      0.032      1.721      0.090      -0.009       0.117
Expectancy    -0.2210      0.010    -22.754      0.000      -0.240      -0.202
==============================================================================
Omnibus:                        0.480   Durbin-Watson:                   0.758
Prob(Omnibus):                  0.787   Jarque-Bera (JB):                0.150
Skew:                          -0.103   Prob(JB):                        0.928
Kurtosis:                       3.117   Cond. No.                     2.40e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.4e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

,df,sum_sq,mean_sq,F,PR(>F)
Literacy,1.0,51.068300,51.068300,5417.263510,2.715190e-61
Expectancy,1.0,4.880946,4.880946,517.764844,1.687004e-31
Residual,61.0,0.575044,0.009427,NaN,NaN


### Greece

In [124]:
res = experiment(data,'GRC','Mortality ~ Expectancy + Literacy')
res.summary()
sm.stats.anova_lm(res)

/tmp/ipykernel_274264/2751998543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Mortality'] = np.log(tmp['Mortality'])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Mortality   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     2265.
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           5.80e-58
Time:                        22:39:26   Log-Likelihood:                 57.487
No. Observations:                  64   AIC:                            -109.0
Df Residuals:                      61   BIC:                            -102.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     22.5501      0.349     64.560      0.000      21.852      23.249
Expectancy    -0.1784      0.008    -23.721      0.000      -0.193      -0.163
Literacy      -0.0744      0.008     -9.177      0.000      -0.091      -0.058
==============================================================================
Omnibus:                        4.575   Durbin-Watson:                   0.445
Prob(Omnibus):                  0.102   Jarque-Bera (JB):                4.310
Skew:                          -0.634   Prob(JB):                        0.116
Kurtosis:                       2.918   Cond. No.                     3.26e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.26e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

,df,sum_sq,mean_sq,F,PR(>F)
Expectancy,1.0,45.307060,45.307060,4446.111527,1.044772e-58
Literacy,1.0,0.858219,0.858219,84.219506,4.282908e-13
Residual,61.0,0.621606,0.010190,NaN,NaN


### Egypt

In [149]:
res = experiment(data,'EGY','Mortality ~ Literacy + Expectancy')
res.summary()
sm.stats.anova_lm(res)

/tmp/ipykernel_274264/2751998543.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Mortality'] = np.log(tmp['Mortality'])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Mortality   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     8899.
Date:                Fri, 29 Mar 2024   Prob (F-statistic):           5.90e-76
Time:                        18:49:35   Log-Likelihood:                 101.27
No. Observations:                  64   AIC:                            -196.5
Df Residuals:                      61   BIC:                            -190.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.7807      0.084     93.000      0.000       7.613       7.948
Literacy      -0.0309      0.001    -32.854      0.000      -0.033      -0.029
Expectancy    -0.0393      0.002    -20.522      0.000      -0.043      -0.035
==============================================================================
Omnibus:                        0.266   Durbin-Watson:                   0.171
Prob(Omnibus):                  0.875   Jarque-Bera (JB):                0.138
Skew:                          -0.113   Prob(JB):                        0.933
Kurtosis:                       2.968   Cond. No.                     1.00e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

,df,sum_sq,mean_sq,F,PR(>F)
Literacy,1.0,45.072382,45.072382,17377.230450,1.244424e-76
Expectancy,1.0,1.092332,1.092332,421.138441,4.484325e-29
Residual,61.0,0.158219,0.002594,NaN,NaN


Let us summarize

| Country | Variable 1 | Variable 2 | $R^2$ |
|---------|------------|------------|-------|
| Turkey  | Literacy   |    GDP     | %97.7 |
| Spain   | Literacy   |    GDP     | %97.7 |
| France  |            |            |       |
| Israel  |   GDP      | Expectancy | %98.1 |
| Italy   | Literacy   | Expectancy | %99.0 |
| Greece  | Expectancy | Literacy   | %98.7 |
| Egypt   | Literacy   | Expectancy | %99.7 |


The order of the variables is also important. Try the models with the order of the important variables reversed, and see that the ANOVA results show that the weight of the most important variable drops.  On the other hand, if you add interaction terms the ANOVA results show no additional drop in the variance (look at the residuals.)

## Q2

Get the following commodity price data from yahoo finance using the `yfinance` library:

- Silver (SI=F)
- Copper (HG=F)
- Platinum (PL=F)
- Gold (GC=F)
- Palladium (PA=F)

1. Write a linear regression model that relates the gold futures in terms of the other precious metals.
2. Analyze the regression results.
3. Does the model improve if we add interaction terms? Explain.
4. Now, do the same for each futures in the list above.

## Q3

Use the *Acoustic Extinguisher Fire Dataset* from Murat Köklü's [data server](https://www.muratkoklu.com/datasets/).

1. Explore the dataset, and project it to 2D space using PCA and LDA. Color the data points using the `STATUS` column.
2. Construct an SVM model to model the `STATUS` column and measure its quality using Accuracy, Precision, Recall, and F-1.
3. Construct a Logistic Regression model to model the `STATUS` column and measure its quality using Accuracy, Precision, Recall, and F-1.
4. Using the LR model, determine which variables affect the most the `STATUS` column.

## Q4

Use the hyperspectral image data (ROSIS sensor data over Pavia Italy) we used for Question 2 from HW1 for this question.

1. Load both the image data and the ground truth data. Reshape the image and name is as `vectors` and the ground truth data as `labels`. 
2. Remove all data points whose label is 0.
3. Write a function that construct a multi-label logistic regression model relating `vectors` to `labels`, and analyzes the accuracy using a correct statistical methodology. Analyze the accuracy results.
4. Now, run a model once over a single training and test set. Report the accuracy, precision, recall, and F1 per label basis. 
5. Repeat (3) and (4) for a multi-label SVM model.
6. Construct confusion matrices over a single run for both LR and SVM, and compare. Present your conclusions.